# Getting Started

You can create vectors using `json_array_to_vector` and then you can calculate distance between to vectors using `json_array_to_vector`:

In [3]:
declare @v1 varbinary(8000) = json_array_to_vector('[1,3,-5]');
declare @v2 varbinary(8000) = json_array_to_vector('[4,-2,-1]');

select 
    vector_distance('euclidean', @v1, @v2) AS euclidean_distance,
    vector_distance('cosine', @v1, @v2) AS cosine_distance,
    vector_distance('dot', @v1, @v2) AS dotproduct_distance;

(1 row affected)

Total execution time: 00:00:00.093

euclidean_distance,cosine_distance,dotproduct_distance
7.0710678118654755,0.8893433329655024,-3


## Store vectors in tables

Vectors can be stored in a table. The function `isvector` can be used to make sure that the stored binary is actually a vector

In [21]:
drop table if exists #vectors
create table #vectors
(
    id int not null identity primary key,
    vector varbinary(8000) not null check (isvector(vector)=1)
);

insert into #vectors (vector)
select 
    json_array_to_vector(a)
from
    ( values 
        ('[4, -2, -1]'),
        ('[1, 3, -5]'),
        ('[7, -8, -10]'),
        ('[4.0, 0.2, -1.1]'),
        ('[0, 0, 0]'),
        ('[10, 10, 10]'),
        ('[-0.1, -0.2, -0.3]')
    ) V(a)
;

select id, vector_to_json_array(vector) as v from #vectors

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.110

id,v
1,"[4.0000000000000000,-2.0000000000000000,-1.0000000000000000]"
2,"[1.0000000000000000,3.0000000000000000,-5.0000000000000000]"
3,"[7.0000000000000000,-8.0000000000000000,-10.0000000000000000]"
4,"[4.0000000000000000,0.2000000029802322,-1.1000000238418579]"
5,"[0.0000000000000000,0.0000000000000000,0.0000000000000000]"
6,"[10.0000000000000000,10.0000000000000000,10.0000000000000000]"
7,"[-0.1000000014901161,-0.2000000029802322,-0.3000000119209290]"


# Query vectors in tables

Vector distance can be used with any valid expression that evaluate to a vector. For example it can be used to calcualte distance between vectors in a table

In [22]:
declare @v varbinary(8000) = json_array_to_vector('[6.4,-2.5,1.3]');
 
select 
    v.id,    
    vector_distance('cosine', @v, v.vector) as distance
from
    #vectors v
where
    vector_distance('cosine', @v, v.vector) < 0.5
order by
    distance


(3 rows affected)

Total execution time: 00:00:00.097

id,distance
1,0.08566878877110973
4,0.18501562892002543
3,0.4924421685441406


# Check vector validity

`isvector` returns 0 (false) if the given binary doesn't contain vector data

In [15]:
select isvector(0x123) as is_vector;

(1 row affected)

Total execution time: 00:00:00.084

is_vector
0


Since there is a check constraint using `isvector` on the sample table, the following insert will error out, as the only way to create a valid vector is via the function `json_array_to_vector`

In [13]:
insert into #vectors (vector) values (0x123)

: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the CHECK constraint "CK__#vectors___vecto__A59E5159". The conflict occurred in database "tempdb", table "#vectors", column 'vector'.

The statement has been terminated.

Total execution time: 00:00:00.082

# Vector Dimensions

In order to calculate distance, vectors must have the same number of dimensions. The following query will fail as the query vector `@v` has 6 dimensions while the stored vectors have only 3.

In [23]:
declare @v varbinary(8000) = json_array_to_vector('[1, 2, 3, 4, 5, 6]');
 
select 
    v.id,    
    vector_distance('cosine', @v, v.vector) as distance
from
    #vectors v
where
    vector_distance('cosine', @v, v.vector) < 0.5
order by
    distance

Total execution time: 00:00:00.088

id,distance


: Msg 42204, Level 16, State 3, Line 3
The input VECTORs have dimension of 6 and 3, respectively, which do not match.